In [23]:
import pandas as pd
import numpy as np
import os
import sys
import regex as re
import json

data_path = "D:/DANG NHI/Cao học/Khai thác văn bản/Đồ án cuối kỳ/data"
# data_path = "C:/Users/Admin/Desktop/DANG NHI/Cao học - Khai thác văn bản/Đồ án cuối kỳ/data"

MAX_LEN = 500
OVERLAP = 100

In [24]:
def preprocess_phrase1(paragraphs):
    for i in range(len(paragraphs)):
        s = paragraphs[i]
        s = re.sub('CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM', '', s)
        s = re.sub('Độc lập - Tự do - Hạnh phúc', '', s)
        s = re.sub(r'\b[\d.,]{5,}\b', 'num', s)
        s = re.sub('[- ]+', ' ', s)
        s = s.strip()
        paragraphs[i] = s

    for i in range(3):
        for i in range(len(paragraphs)):
            if len(paragraphs[i]) < 1:
                paragraphs.pop(i)
                break
    return paragraphs

def count_words(input_text):
    return len(input_text.split())

def markdown_chunking(paragraphs, max_len=512, overlap=100):
    lst_chunks = []
    current_chunk = ''
    for p in paragraphs:
        if count_words(current_chunk) + count_words(p) <= max_len:
            if len(current_chunk) == 0:
                current_chunk = p
            else:
                current_chunk = "\n".join([current_chunk, p])
        else:
            lst_chunks.append(current_chunk)
            current_chunk = " ".join(current_chunk.split(" ")[-100:]) + "\n" + p

    lst_chunks.append(current_chunk)
    return lst_chunks

In [26]:
files = os.listdir(data_path)
data_stored = []

# Test 1 document

In [17]:
file = files[0]
file

'BaoHiem.json'

In [18]:
with open(data_path + '/' + file, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [13]:
doc = data[0]

In [21]:
content = doc['content']
content = "\n".join(content)

In [22]:
print(content)

HỘI ĐỒNG NHÂN DÂN TỈNH KHÁNH HÒA -------
CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM Độc lập - Tự do - Hạnh phúc ---------------
Số: 06/2024/NQ-HĐND
Khánh Hòa, ngày 12 tháng 7 năm 2024
NGHỊ QUYẾT
VỀ VIỆC BAN HÀNH MỨC GIÁ DỊCH VỤ KHÁM BỆNH, CHỮA BỆNH KHÔNG THUỘC PHẠM VI THANH TOÁN CỦA QUỸ BẢO HIỂM Y TẾ MÀ KHÔNG PHẢI LÀ DỊCH VỤ KHÁM BỆNH, CHỮA BỆNH THEO YÊU CẦU TẠI CÁC CƠ SỞ KHÁM BỆNH, CHỮA BỆNH CỦA NHÀ NƯỚC TRÊN ĐỊA BÀN TỈNH KHÁNH HÒA
HỘI ĐỒNG NHÂN DÂN TỈNH KHÁNH HÒA KHÓA VII, KỲ HỌP THỨ 14
Căn cứ Luật Tổ chức chính quyền địa phương ngày 19 tháng 6 năm 2015;
Căn cứ Luật sửa đổi, bổ sung một số điều của Luật Tổ chức Chính phủ và Luật tổ chức chính quyền địa phương ngày 22 tháng 11 năm 2019;
Căn cứ Luật Ban hành văn bản quy phạm pháp luật ngày 22 tháng 06 năm 2015 ;
Căn cứ Luật sửa đổi, bổ sung một số điều của Luật Ban hành văn bản quy phạm pháp luật ngày 18 tháng 6 năm 2020;
Căn cứ Luật Khám bệnh, chữa bệnh ngày 09 tháng 01 năm 2023;
Căn cứ Luật Giá ngày 20 tháng 06 năm 2012 ;
Căn cứ Nghị định số

# Run whole data folder

In [4]:
for file in files:
    print(file)
    field_chunks = []
    if file.endswith('.json'):
        with open(data_path + '/' + file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        for dct in data:
            paragraphs = dct['content']
            paragraphs = preprocess_phrase1(paragraphs)
            chunks = markdown_chunking(paragraphs, MAX_LEN, OVERLAP)
            field_chunks.append(chunks)

    data_stored.append(field_chunks)

BaoHiem.json


KeyboardInterrupt: 

# Find by indices

# Random bình thường

In [ ]:
import random

import numpy as np
from itertools import combinations

def get_very_positive_pairs(data_stored, max_pairs= 500000):
    pairs = []
    for i in range(len(data_stored)):
        for j in range(len(data_stored[i])):
            chunks = data_stored[i][j]
            for k in range(len(chunks)):
                for l in range(k+1, len(chunks)):
                    pairs.append(((i,j,k), (i,j,l), 1))
    return random.sample(pairs, max_pairs)

def get_low_positive_pairs(data_stored):
    pairs = []
    for i in range(len(data_stored)):
        l = len(data_stored[i])
        for j in range(l):
            for k in range(j+1, l):
                a = random.randrange(len(data_stored[i][j]))
                b = random.randrange(len(data_stored[i][k]))
                pairs.append((i, j, a) , (i,k,b), 0.5)
    return pairs


def get_negative_pairs(data_stored, mean_per_field_pair=100000):
    pairs = []
    l = len(data_stored)
    for i in range(l):
        for j in range(i+1, l):
            indices1 = [(i, a, b) for a in range(len(data_stored[i])) for b in range(len(data_stored[i][a]))]
            indices2 = [(j, a, b) for a in range(len(data_stored[j])) for b in range(len(data_stored[j][a]))]

            a = np.random.choice(len(indices1), mean_per_field_pair)
            b = np.random.choice(len(indices2), mean_per_field_pair)

            for x, y in zip(a, b):
                pairs.append((indices1[x], indices2[y], 0))
    return pairs

In [57]:
random.randrange(10)

4

In [ ]:
sentence1 = []
sentence2 = []
scores = []
for pair in pairs:
    (i, j, k), (x, y, z), score = pair
    sentence1.append(data_stored[i][j][k])
    sentence2.append(data_stored[x][y][z])
    scores.append(score)

In [ ]:
dataframe = pd.DataFrame({'sentence1': sentence1, 'sentence2': sentence2, 'score': scores})

In [ ]:
data.to_csv('data.csv', index=False)

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training, validation, and test sets
train_data, test_data = train_test_split(pairs, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

In [58]:
class CustomDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        (i,j,k), (x,y,z), scores = self.pairs[idx]
        sentence1 = data_stored[i][j][k]
        sentence2 = data_stored[x][y][z]
        return sentence1, sentence2, scores

NameError: name 'Dataset' is not defined

In [ ]:
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [61]:
import torch
torch.cosine_similarity(torch.tensor([1.0, 2.0, 3.0]).unsqueeze(0), torch.tensor([1.0, 2.0, 3.0]).unsqueeze(0))

tensor([1.0000])